In [1]:
from sqlalchemy import create_engine, Table, MetaData, Column
from sqlalchemy import Integer, String, DateTime, Boolean
from sqlalchemy import ForeignKey, ForeignKeyConstraint

In [2]:
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)

In [3]:
USER = "postgres"
PASSWORD = "postgres"
HOST = ""
DATABASE = "swlab"


engine = create_engine("postgresql+psycopg2://{}:{}@{}/{}".format(USER, PASSWORD, HOST, DATABASE), client_encoding="utf8")

In [4]:
metadata = MetaData()
metadata.bind = engine

In [5]:
types = Table(
    'types', metadata,
    Column('type_id', Integer, primary_key=True),
    Column('name', String(45))
)

In [6]:
features = Table(
    'features', metadata,
    Column('feature_id', Integer, primary_key=True),
    Column('namespace', String(255)),
    Column('type_id', Integer, nullable=False),
    ForeignKeyConstraint(['type_id'], [types.c.type_id], 
                         name='fk_features_types',
                         ondelete='NO ACTION', onupdate='NO ACTION')
)

In [7]:
datasets = Table(
    'datasets', metadata,
    Column('feature_id', Integer, primary_key=True),
    Column('meta', String(45)),
    Column('meta_id', String(45)),
    Column('meta_url', String(255)),
    Column('name', String(45)),
    Column('url', String(255)),
    Column('created_at', DateTime),
    Column('modified_at', DateTime),
    ForeignKeyConstraint(['feature_id'], [features.c.feature_id], 
                         name='fk_datasets_features',
                         ondelete='NO ACTION', onupdate='NO ACTION')
)

In [8]:
resources = Table(
    'resources', metadata,
    Column('resource_id', Integer, primary_key=True),
    Column('format', String(45)),
    Column('url', String(255)),
    Column('source', String(255)),
    Column('description', String(255)),
    Column('is_online', Boolean),
    Column('feature_id', Integer),
    ForeignKeyConstraint(['feature_id'], [datasets.c.feature_id], 
                         name='fk_resources_datasets',
                         ondelete='NO ACTION', onupdate='NO ACTION')
)

In [9]:
dataset_features = Table(
    'dataset_features', metadata,
    Column('ds_feature_id', Integer, primary_key=True),
    Column('ft_feature_id', Integer, primary_key=True),
    Column('count', Integer),
    Column('resource_id', Integer),
    ForeignKeyConstraint(['ds_feature_id'], [datasets.c.feature_id],
                         name='fk_dataset_features_datasets',
                         ondelete='NO ACTION', onupdate='NO ACTION'),
    ForeignKeyConstraint(['ft_feature_id'], [features.c.feature_id],
                         name='fk_dataset_features_features',
                         ondelete='NO ACTION', onupdate='NO ACTION'),
    ForeignKeyConstraint(['resource_id'], [resources.c.resource_id],
                         name='fk_dataset_features_resources',
                         ondelete='NO ACTION', onupdate='NO ACTION')
)

In [10]:
if dataset_features.exists():
    dataset_features.drop()
if resources.exists():
    resources.drop()
if datasets.exists():
    datasets.drop()
if features.exists():
    features.drop()
if types.exists():
    types.drop()
types.create()
features.create()
datasets.create()
resources.create()
dataset_features.create()

INFO:sqlalchemy.engine.base.Engine:select version()
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:select current_schema()
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:show standard_conforming_strings
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
INFO:sqlalchemy.engine.base.Engine:{'name': 'dataset_features'}
INFO:sqlalchemy.engine.base.Engine:
DROP TABLE dataset_features
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:COMMIT
INFO:sqlalchemy.engine.base.Engine:select relname from pg_